# Results from 7/30 licensing program

At the end of last year we changed our demo licensing program with the goals of:

1. being able to better track sales evaluations
2. having a mechanism to give customers and academics temporary licenses
3. improving the ability of new users to get started right away

This was difficult to evaluate through the holidays as they interfere with the standard usage patters but now that we're in the new year we want to start tracking this program and understanding whether it is achieving its goals.

In the beginning we ran into some issues with academic users so I will only analyze the data from April 1 onwards.

## Overall Summary

In [23]:
print('This analysis covers licenses generated between', start_date, 'and', end_date)
print('{} 7-day licenses were generated'.format(len(df[df.period==7].email.unique())))
print('Of those, {} were converted to something else, {} just expired'.format(num_cvt, len(temp_users)-num_cvt))

This analysis covert licenses generated between 2017-04-01 and 2017-06-30
524 7-day licenses were generated
Of those, 244 were converted to something else, 271 just expired


That include academic licenses though.  Once we will exclude academic users since our focus is on commercial evaluations.

In [25]:
print('Number of commercial demoers is {}'.format(len(demos.email.unique())))

Number of commercial demoers is 139


## Self Identification

These evaluators, at registration time, identify their interests.  Let's look at how they breakdown.

In [26]:
print('Breakdown by Application Interest')
demo_info.applicationinterest.value_counts()

Breakdown by Application Interest


RF/Microwave Circuits        118
RF/Communications Systems      9
Academic                       7
Name: applicationinterest, dtype: int64

In [27]:
print('Breakdown by Design Technology')
demo_info.designtechnology.value_counts()

Breakdown by Design Technology


4:PCB                                 58
4:Other                               24
4:RFIC                                16
4:Modules                             10
4:MMIC                                10
1:Classroom Instruction (Labs)         4
3:Circuit and System Co-Simulation     3
1:Mix of the Above                     3
3:Other                                3
3:Radar and Phased-Array               2
3:5G/Communication Systems             1
Name: designtechnology, dtype: int64

In [28]:
print('Breakdown by Design Focus')
demo_info.designfocus.value_counts()

Breakdown by Design Focus


4:Other                         45
4:Amplifiers                    36
4:Filters                       27
4:Mixers                         5
4:Oscillators                    5
1:RF/Microwave Circuits          4
3:End-to-End Architecture        3
3:RF/Microwave Front-End         3
1:Mix of the Above               2
3:Baseband Signal Processing     2
1:RF/Communications Systems      1
3:Other                          1
Name: designfocus, dtype: int64

## Internal Categorization

We also have some internal variables we can categorize them by.

In [31]:
print('Breakdown by Rep Territory')
demo_info.territory.value_counts()

Breakdown by Rep Territory


AWR-CN,BLC,CRI,UNI        17
AWR-DE                    11
ANE                       10
AWR-SE                     9
IDA                        9
AWR-NW                     8
AWR-UK                     8
AWR-NE                     8
WLB                        8
AWR-RU,MIC,PEL,ROD,SOF     6
AWR-SW                     5
AWR-SP                     5
AWR-AU                     4
AWR-FR                     3
SMC                        3
AWR-KR                     3
AWR-IT                     3
CONTROL                    2
AWR-CE                     2
AWR-TSM,MTP                1
VIM                        1
AWR-EU                     1
RSA                        1
AWR-AP                     1
NI-PL                      1
AWR-AP,MTP                 1
AWR-JP,NI-JP               1
SOF                        1
AWR-SP,DSC                 1
Name: territory, dtype: int64

In [32]:
print('Breakdown by Sales Person')
demo_info.gen_by.value_counts()

Breakdown by Sales Person


nbasha      34
pxu         18
firekonh    10
smaynard     9
uksania      8
ukwallaa     8
filambem     7
vjensen      7
dniven       7
lnakamur     6
rmcadam      6
aoh          3
pfagerud     3
jgebhard     2
cebertow     2
grpelhao     2
ntadokor     1
ukkhant      1
Name: gen_by, dtype: int64

---
# Analysis Details - Stop Reading Here

In [1]:
import sys
sys.path.append('/Users/dane/src/datatools')
import pandas as pd
from dtutil.util import init_cdb
from dtutil.sqltools import query_to_dataframe

In [2]:
# look only at users that have received a 7-day license and get all their licenses
start_date = '2017-04-01'
end_date = '2017-06-30'
conn = init_cdb()
cursor = conn.cursor()
query = """
select email, hostid, cast(gen_date as date) as gen, period, serial_num, description 
from lds_lic_history
	inner join lds_lic_types 
		on lds_lic_history.lictypeid = lds_lic_types.lictypeid
where email in (select distinct email from lds_lic_history where period=7 
                and gen_date > '{0}')
	and gen_date between '{0}' and '{1}'
order by email, gen
""".format(start_date, end_date)
df = query_to_dataframe(cursor, query)
print('There are {} records for {} users'.format(len(df), len(df.email.unique())))

There are 776 records for 524 users


In [22]:
len(df[df.period==7].email.unique())

524

In [4]:
temp = df[df.description=='Temp']
non_temp = df[df.period > 7]
temp_users = temp.email.unique()
print('There were {} temp licenses for {} users generated'.format(len(temp), len(temp_users)))

There were 521 temp licenses for 515 users generated


In [5]:
converted_users = non_temp[non_temp.email.isin(temp_users)]
num_cvt = len(converted_users)
print('Of those, {} were converted to something else, {} just expired'.format(num_cvt, len(temp_users)-num_cvt))

Of those, 244 were converted to something else, 271 just expired


In [6]:
converted_users.description.value_counts()

Demo           157
Educational     87
Name: description, dtype: int64

In [7]:
# let's drop the educational
demos = converted_users[converted_users.description != 'Educational']
demos.head(5)

,email,hostid,gen,period,serial_num,description
4,13608333720@163.com,28E347AD2028,2017-05-15,30,90064383,Demo
6,15211070500@163.com,201701261804,2017-04-28,30,90064035,Demo
9,18729098353@163.com,80A5895DE8A9,2017-04-28,30,90064126,Demo
11,1945055657@qq.com,847beb1cc68b,2017-04-28,30,90064188,Demo
13,1952729015@qq.com,DISK_SERIAL_NUM=524688cf,2017-04-28,30,90064104,Demo


In [8]:
# that does not get all the academic
from dtutil.util import EmailChecker
checker = EmailChecker()

l = []
for email in demos.email.unique():
    if checker.email_type(email) == 'academic':
        l.append(email)
demos = demos[~demos.email.isin(l)]
print('Number of actual demoers is {}'.format(len(demos.email.unique())))

Number of actual demoers is 139


In [14]:
demo_emails = demos.email.unique()
query = """
select CFSrepterritory as Territory, FullName, EmailAddress1 as Email, CompanyName, serial_num, gen_by,
       df.value as DesignFocus, ai.value as ApplicationInterest, dt.value as DesignTechnology
from lds_lic_history as h
    inner join Applied_Wave_Research_MSCRM.dbo.LeadBase as lb  on h.leadid = lb.leadid
    inner join Applied_Wave_Research_MSCRM.dbo.LeadExtensionBase as leb on lb.leadid = leb.leadid
	inner join Applied_Wave_Research_MSCRM.dbo.stringmap as df
		on df.attributename = 'awr_designfocus' and df.attributevalue = AWR_designfocus
	inner join Applied_Wave_Research_MSCRM.dbo.stringmap as ai
		on ai.attributename = 'awr_applicationinterest' and ai.attributevalue = AWR_applicationinterest
	inner join Applied_Wave_Research_MSCRM.dbo.stringmap as dt
		on dt.attributename = 'awr_designtechnology' and dt.attributevalue = AWR_designtechnology
where EmailAddress1 in ( {} )
and period != 7
""".format(','.join(["'{}'".format(x) for x in demo_emails]))
demo_info = query_to_dataframe(cursor, query)
demo_info.head()

,territory,fullname,email,companyname,serial_num,gen_by,designfocus,applicationinterest,designtechnology
0,AWR-FR,"Plet, Sullivan",sullivan.plet@macom.com,Macom,90064469,pfagerud,4:Amplifiers,RF/Microwave Circuits,4:MMIC
1,AWR-KR,"Kim, Eunseob",keslloddy@gmail.com,None,90064366,aoh,4:Amplifiers,RF/Microwave Circuits,4:MMIC
2,"AWR-CN,BLC,CRI,UNI","pi, liming",piliming@ponted.com,"SHENZHEN PONTED BIO-MED CO.,LTD.",90064219,pxu,4:Mixers,RF/Microwave Circuits,4:MMIC
3,IDA,"Mallesh, Tvn",helatesting@hela.in,HBL Elta Avionics Systems Pvt. Ltd. - Hyderabad,90064340,nbasha,4:Amplifiers,RF/Microwave Circuits,4:MMIC
4,AWR-SW,"Chamberlin, Richard",cham314@hotmail.com,None,90064038,lnakamur,4:Amplifiers,RF/Microwave Circuits,4:MMIC
